In [369]:
import requests
import pandas as pd

years = range(2005, 2024)  # From 2010 to 2022
data = []

# Replace with your API key
API_KEY = '59951fa51ae0813c57d78e2f22413294efb566a6'

# DP03_0003E, DP03_0005E
for year in years:
    if year > 2010:
        # For years greater than 2010, make a single API call to get all data
        if(year != 2020):
            url = f"https://api.census.gov/data/{year}/acs/acs1?get=NAME,B01001_001E,B19013_001E,B23025_005E,B23025_003E,B01002_001E,B07001_001E&for=place:*&key={API_KEY}"
        else:
            url = f"https://api.census.gov/data/{year}/acs/acs5?get=NAME,B01001_001E,B19013_001E,B23025_005E,B23025_003E,B01002_001E,B07001_001E&for=place:*&key={API_KEY}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data for year {year}")
            continue
        json_data = response.json()
        for row in json_data[1:]:  # Skip the header row
            city_name = row[0]
            try:
                population = int(row[1])
                mean_income = float(row[2])
                unemployed = float(row[3])
                total_labor_force = float(row[4])
                median_age = float(row[5])
                geo_mobility = float(row[6])
            except:
                population = 0

            # Filter for population greater than 250,000
            if population > 100000:
                data.append({
                    'Year': year,
                    'City': city_name,
                    'Total Population': population,
                    'Mean Household Income': mean_income,
                    'Unemployment Rate': unemployed/total_labor_force*100,
                    'Median Age': median_age,
                    'Geographic Mobility': geo_mobility,
                })
    else:
        # For years <= 2010, make two API calls: one for general data and one for unemployment data
        url_general = f"https://api.census.gov/data/{year}/acs/acs1?get=NAME,B01001_001E,B19013_001E,B01002_001E,B07001_001E&for=place:*&key={API_KEY}"
        url_unemployment = f"https://api.census.gov/data/{year}/acs/acs1/profile?get=NAME,DP03_0003E,DP03_0005E&for=place:*&key={API_KEY}"
        
        response_general = requests.get(url_general)
        response_unemployment = requests.get(url_unemployment)
        
        if response_general.status_code != 200 or response_unemployment.status_code != 200:
            print(f"Failed to fetch data for year {year}")
            continue
        
        json_data_general = response_general.json()
        json_data_unemployment = response_unemployment.json()
        
        # Create a lookup for unemployment data by city name
        unemployment_dict = {}
        for row in json_data_unemployment[1:]:  # Skip the header row
            city_name = row[0]
            try:
                unemployed = float(row[1])
                total_labor_force = float(row[2])
                unemployment_dict[city_name] = (unemployed, total_labor_force)
            except:
                unemployment_dict[city_name] = (0, 0)
        
        for row in json_data_general[1:]:  # Skip the header row
            city_name = row[0]
            try:
                population = int(row[1])
                mean_income = float(row[2])
                median_age = float(row[3])
                geo_mobility = float(row[4])
                unemployed, total_labor_force = unemployment_dict.get(city_name, (0, 0))
            except:
                population = 0

            # Filter for population greater than 100,000
            if population > 100000:
                if total_labor_force > 0:  # Prevent division by zero
                    unemployment_rate = unemployed / total_labor_force 
                else:
                    unemployment_rate = None
                
                data.append({
                    'Year': year,
                    'City': city_name,
                    'Total Population': population,
                    'Mean Household Income': mean_income,
                    'Unemployment Rate': unemployment_rate,
                    'Median Age': median_age,
                    'Geographic Mobility': geo_mobility,
                })

# Create a DataFrame
df = pd.DataFrame(data)
state_abbreviation_map = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI',
    'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Step 2: Replace state names with abbreviations in the 'City' column
def replace_state_names(city_name):
    for state, abbrev in state_abbreviation_map.items():
        if state in city_name:
            return city_name.replace(state, abbrev)
    return city_name
pivot_income = df.pivot(index='City', columns='Year', values='Mean Household Income')
df_income = pivot_income
df_income = df_income.dropna()
# cleaning out stuff
df_income.reset_index(inplace=True)
df_income = df_income[~df_income['City'].str.contains("Puerto Rico", case=False, na=False)]
df_income['City'] = df_income['City'].str.replace(r' \bcity\b', '', regex=True)
df_income['City'] = df_income['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_income['City'] = df_income['City'].str.replace(r' \bCDP\b', '', regex=True)
df_income['City'] = df_income['City'].str.strip()
df_income['City'] = df_income['City'].apply(replace_state_names)
df_income.reset_index(inplace=True)
df_income = df_income.drop(columns=['index'], errors='ignore')
df_income


Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Albuquerque, NM",41820.0,43021.0,43677.0,46437.0,44594.0,46079.0,43606.0,45373.0,48357.0,46433.0,47096.0,50522.0,50456.0,51099.0,55567.0,53936.0,58512.0,64757.0,67907.0
1,"Anaheim, CA",52158.0,55720.0,57059.0,61128.0,55154.0,54157.0,56858.0,55464.0,57550.0,62198.0,63104.0,64464.0,70395.0,76154.0,76075.0,76723.0,81747.0,85133.0,84872.0
2,"Anchorage, AK",61217.0,63656.0,68726.0,75637.0,72832.0,72128.0,72813.0,72575.0,79045.0,76337.0,78662.0,85634.0,79166.0,83648.0,82716.0,84813.0,86654.0,100751.0,94437.0
3,"Arlington, VA",80433.0,87350.0,94876.0,101171.0,96218.0,94986.0,100735.0,100474.0,102501.0,109266.0,106768.0,110388.0,117237.0,122394.0,119755.0,122604.0,125651.0,132380.0,140219.0
4,"Arlington, TX",48992.0,50582.0,50447.0,52777.0,50938.0,48752.0,50655.0,51285.0,51400.0,52141.0,53487.0,54272.0,57083.0,63091.0,61716.0,63351.0,59889.0,70433.0,69208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,"Tulsa, OK",35966.0,36137.0,38270.0,39657.0,38426.0,38220.0,40123.0,40440.0,41495.0,42803.0,43322.0,43134.0,45894.0,47583.0,49158.0,49474.0,51008.0,54040.0,56821.0
83,"Vancouver, WA",40743.0,45884.0,47989.0,50990.0,45701.0,47307.0,48909.0,45718.0,47379.0,51478.0,51988.0,56615.0,61079.0,60988.0,66679.0,63617.0,69993.0,76799.0,80618.0
84,"VA Beach, VA",58545.0,61333.0,61462.0,65776.0,59298.0,64212.0,64614.0,61626.0,62855.0,68816.0,67281.0,71117.0,72586.0,77059.0,79054.0,78136.0,81634.0,83245.0,91141.0
85,"Washington, DC",47221.0,51847.0,54317.0,57936.0,59290.0,60903.0,63124.0,66583.0,67572.0,71648.0,75628.0,75506.0,82372.0,85203.0,92266.0,90842.0,90088.0,101027.0,108210.0


In [361]:
pivot_unem = df.pivot(index='City', columns='Year', values='Unemployment Rate')
df_unem = pivot_unem
df_unem = df_unem.dropna()
# cleaning out stuff
df_unem.reset_index(inplace=True)
df_unem = df_unem[~df_unem['City'].str.contains("Puerto Rico", case=False, na=False)]
df_unem['City'] = df_unem['City'].str.replace(r' \bcity\b', '', regex=True)
df_unem['City'] = df_unem['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_unem['City'] = df_unem['City'].str.replace(r' \bCDP\b', '', regex=True)
df_unem['City'] = df_unem['City'].str.strip()
df_unem['City'] = df_unem['City'].apply(replace_state_names)
df_unem.reset_index(inplace=True)
df_unem = df_unem.drop(columns=['index'], errors='ignore')
df_unem

Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Albuquerque, NM",14.533702,18.786019,19.911130,19.150972,13.877965,12.436434,10.542100,8.817330,7.260191,8.308843,6.681222,5.212342,4.805098,5.734425,5.082790,5.594741,6.105605,4.188383,4.337847
1,"Atlanta, GA",10.534444,10.190236,10.807414,11.776196,7.966137,6.696835,13.222761,14.661379,11.253640,10.482968,7.382863,7.278972,6.648207,5.659872,4.996723,6.626817,5.200898,4.757105,5.678080
2,"Austin, TX",14.198369,16.347289,19.592060,19.434976,13.391173,11.853359,8.251289,7.106502,5.853625,4.469576,4.170990,3.476909,3.537583,3.772958,3.428839,4.362911,5.291025,3.045554,3.950272
3,"Baltimore, MD",8.795519,9.346614,9.931212,10.222290,7.826814,6.936303,16.148262,15.486366,11.114411,11.819685,10.074000,8.923227,7.569993,6.761844,7.310009,7.555764,7.922479,5.496045,5.053969
4,"Boston, MA",11.037364,15.563555,12.151404,13.033743,8.894487,7.781088,11.563092,9.604663,8.080456,8.327116,6.673833,7.570566,6.710638,6.867020,4.410932,6.880256,8.022814,5.225530,4.171435
5,"Charlotte, NC",12.053908,13.637751,13.447170,15.167830,9.113040,7.493992,11.286430,10.537584,10.168049,9.224919,7.340828,5.412054,4.623674,4.183091,3.603295,4.948711,5.031209,3.285272,3.527264
6,"Chicago, IL",9.058758,10.159389,10.919533,10.602607,7.641775,6.742210,14.097142,13.663928,12.665761,10.912343,9.496046,8.148354,8.304362,8.135736,6.289193,8.102961,10.103574,6.436242,6.440380
7,"CO Springs, CO",14.340945,17.183562,14.319420,15.405415,11.413189,9.215859,11.689782,9.409037,9.168744,7.255789,6.749931,5.709215,6.009308,4.306983,5.425079,5.794375,6.290016,4.137689,4.506438
8,"Columbus, OH",12.674405,12.393934,13.996620,15.437808,10.593032,8.129315,10.746960,8.545421,7.212225,6.416014,6.551108,6.159773,5.079323,5.045730,4.629965,5.417690,6.229611,4.371886,4.388452
9,"Dallas, TX",11.630534,12.257335,15.741236,16.425376,10.705989,9.868313,9.779732,9.275216,7.912220,6.628744,5.185364,4.682171,5.226705,4.318155,4.611744,4.902161,6.029384,4.017421,4.411595


In [362]:
pivot_age = df.pivot(index='City', columns='Year', values='Median Age')
df_age = pivot_age
df_age = df_age.dropna()
# cleaning out stuff
df_age.reset_index(inplace=True)
df_age = df_age[~df_age['City'].str.contains("Puerto Rico", case=False, na=False)]
df_age['City'] = df_age['City'].str.replace(r' \bcity\b', '', regex=True)
df_age['City'] = df_age['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_age['City'] = df_age['City'].str.replace(r' \bCDP\b', '', regex=True)
df_age['City'] = df_age['City'].str.strip()
df_age['City'] = df_age['City'].apply(replace_state_names)
df_age.reset_index(inplace=True)
df_age = df_age.drop(columns=['index'], errors='ignore')
df_age

Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Albuquerque, NM",35.8,35.3,35.2,35.5,34.0,35.2,35.5,35.3,35.8,35.7,36.0,36.5,37.0,37.2,37.3,37.1,38.9,38.4,39.2
1,"Atlanta, GA",34.7,35.0,35.0,35.7,33.4,33.2,32.7,33.1,33.1,33.5,33.8,33.0,33.3,33.2,33.3,33.2,34.3,34.0,34.5
2,"Austin, TX",31.4,31.2,31.4,31.7,31.2,30.9,31.3,31.7,32.0,32.6,32.7,32.7,33.4,33.6,33.9,33.7,34.4,34.5,34.9
3,"Baltimore, MD",35.7,35.6,35.4,35.5,34.0,34.4,34.2,34.2,34.6,34.6,34.7,34.9,35.3,35.5,35.9,35.5,36.1,36.5,36.5
4,"Boston, MA",33.1,33.6,33.4,33.3,32.0,30.8,30.9,31.0,31.7,31.7,31.8,32.1,32.3,32.1,32.6,32.4,33.0,33.3,33.7
5,"Charlotte, NC",33.8,34.7,34.3,35.0,33.3,33.2,33.7,33.3,33.3,33.8,34.3,34.0,34.4,34.7,34.2,34.4,34.8,34.5,34.7
6,"Chicago, IL",33.1,33.6,34.1,34.0,32.9,32.8,33.0,33.6,33.7,33.9,34.2,34.4,34.6,34.9,35.2,34.8,35.8,35.8,36.4
7,"CO Springs, CO",34.0,34.3,35.4,34.8,35.5,35.1,34.2,34.5,34.5,34.6,34.8,34.6,34.7,34.6,35.6,34.7,35.6,36.0,36.4
8,"Columbus, OH",32.1,32.5,32.5,32.5,31.5,31.6,31.6,32.0,32.3,31.9,32.5,32.3,32.2,32.3,32.4,32.4,33.1,33.3,33.4
9,"Dallas, TX",31.9,31.9,31.9,31.9,31.6,31.6,31.6,32.4,32.1,32.4,32.6,32.7,32.4,33.3,32.9,32.9,33.7,33.1,33.5


In [363]:
pivot_geomob = df.pivot(index='City', columns='Year', values='Geographic Mobility')
df_geomob = pivot_geomob
df_geomob = df_geomob.dropna()
# cleaning out stuff
df_geomob.reset_index(inplace=True)
df_geomob = df_geomob[~df_geomob['City'].str.contains("Puerto Rico", case=False, na=False)]
df_geomob['City'] = df_geomob['City'].str.replace(r' \bcity\b', '', regex=True)
df_geomob['City'] = df_geomob['City'].str.replace(r' \bmunicipality\b', '', regex=True)
df_geomob['City'] = df_geomob['City'].str.replace(r' \bCDP\b', '', regex=True)
df_geomob['City'] = df_geomob['City'].str.strip()
df_geomob['City'] = df_geomob['City'].apply(replace_state_names)
df_geomob.reset_index(inplace=True)
df_geomob = df_geomob.drop(columns=['index'], errors='ignore')
df_geomob

Year,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Albuquerque, NM",482442.0,485982.0,505490.0,506706.0,519532.0,540574.0,544579.0,548340.0,549629.0,550317.0,552275.0,553849.0,551049.0,553759.0,553804.0,554784.0,558997.0,554890.0,555391.0
1,"Atlanta, GA",388422.0,438135.0,425366.0,447703.0,532685.0,416670.0,427164.0,438201.0,441997.0,449800.0,460327.0,467099.0,481420.0,491120.0,501947.0,492410.0,491005.0,494738.0,505496.0
2,"Austin, TX",665672.0,706505.0,734639.0,763919.0,777744.0,782461.0,809464.0,831672.0,874143.0,900040.0,919721.0,934294.0,938330.0,953463.0,967577.0,954333.0,953064.0,966426.0,970338.0
3,"Baltimore, MD",600543.0,622612.0,627273.0,628349.0,628138.0,612017.0,610849.0,612602.0,612995.0,615275.0,613552.0,607542.0,603746.0,595926.0,586469.0,595287.0,568926.0,563521.0,558241.0
4,"Boston, MA",511840.0,565420.0,606616.0,603998.0,636866.0,615629.0,618046.0,631408.0,638500.0,648438.0,661446.0,664168.0,676287.0,688679.0,687036.0,681962.0,646938.0,643455.0,645761.0
5,"Charlotte, NC",592140.0,640115.0,663081.0,673310.0,692387.0,725808.0,741987.0,762800.0,784272.0,798163.0,815490.0,830055.0,849948.0,862010.0,872125.0,862707.0,869445.0,885833.0,899656.0
6,"Chicago, IL",2655958.0,2702974.0,2697424.0,2698979.0,2804553.0,2659970.0,2670397.0,2675954.0,2684006.0,2687174.0,2685744.0,2667356.0,2684432.0,2672528.0,2659927.0,2665195.0,2667710.0,2635373.0,2638357.0
7,"CO Springs, CO",370441.0,393914.0,383723.0,390011.0,394498.0,413870.0,420732.0,426130.0,433584.0,440409.0,449948.0,458692.0,457883.0,466933.0,472541.0,466178.0,478717.0,480005.0,482018.0
8,"Columbus, OH",680756.0,705847.0,719362.0,725316.0,760540.0,778002.0,786775.0,798718.0,809934.0,823548.0,836248.0,849806.0,868634.0,883234.0,887026.0,876171.0,895229.0,895858.0,897432.0
9,"Dallas, TX",1121138.0,1170743.0,1217815.0,1200940.0,1272539.0,1184310.0,1206334.0,1222427.0,1242023.0,1260677.0,1278532.0,1296979.0,1320642.0,1328601.0,1321238.0,1319420.0,1270447.0,1279926.0,1287866.0


In [364]:
houseprices = pd.read_csv("../data/house_price.csv") 
housedata = houseprices[["RegionName", "2005-01-31","2006-01-31","2007-01-31", "2008-01-31", "2009-01-31", "2010-01-31", "2011-01-31", "2012-01-31", "2013-01-31", "2014-01-31", "2015-01-31", "2016-01-31", "2017-01-31", "2018-01-31", "2019-01-31", "2020-01-31", "2020-01-31", "2021-01-31", "2022-01-31", "2023-01-31"]]
housedata = housedata.sort_values(by='RegionName')
uniquecities = df_unem["City"].unique()
housedata = housedata[housedata["RegionName"].isin(uniquecities)]
housedata.reset_index(inplace=True)
housedata= housedata.drop(columns=['index'], errors='ignore')
housedata.columns = [col[:4] for col in housedata.columns]
housedata.rename(columns={'Regi': 'City'}, inplace=True)
housedata = housedata.dropna()
# uniquecities = housedata["City"].unique()
housedata.reset_index(inplace=True)
housedata= housedata.drop(columns=['index'], errors='ignore')

In [365]:
housedata

,City,2005,2006,2007,2008,2009,2010,2011,2012,2013,...,2015,2016,2017,2018,2019,2020,2020,2021,2022,2023
0,"Albuquerque, NM",153134.096654,170259.007619,193305.919721,200215.211429,187486.929321,180903.756614,171329.029725,164757.245024,163877.783060,...,169546.379112,176038.325622,183130.699662,189542.195949,1.980266e+05,2.129678e+05,2.129678e+05,2.410663e+05,2.805876e+05,3.081899e+05
1,"Atlanta, GA",179505.707068,186545.601636,192300.481437,191047.167478,171838.828357,152781.210858,139138.933143,125489.469946,131061.115550,...,169023.933257,183060.400400,196831.953236,213850.991653,2.323420e+05,2.448993e+05,2.448993e+05,2.713745e+05,3.335247e+05,3.669642e+05
2,"Austin, TX",186656.878020,192941.385211,199709.993663,207084.238475,201617.906762,198154.577123,192188.468231,190162.573456,201536.716014,...,248561.105165,271392.931472,287958.928699,298873.135564,3.083495e+05,3.223582e+05,3.223582e+05,3.667843e+05,4.954505e+05,4.995716e+05
3,"Baltimore, MD",240788.263853,287923.885227,308939.030866,304359.493716,278858.068520,259095.858290,245870.769189,232832.506267,240968.189706,...,259239.082824,265118.803553,271451.149002,279396.779775,2.869112e+05,2.966322e+05,2.966322e+05,3.193128e+05,3.489373e+05,3.633435e+05
4,"Boston, MA",382715.968493,398584.115753,377807.712517,362087.048748,335987.747593,329631.904785,328532.876118,314696.995585,329627.266928,...,375171.911811,402685.595129,428359.342497,457286.235943,4.788747e+05,4.908629e+05,4.908629e+05,5.407756e+05,5.902593e+05,6.247166e+05
5,"Charlotte, NC",158158.778134,163575.501056,171928.977411,179513.455323,171800.455853,163543.225698,154086.630300,145167.451962,149210.006802,...,167340.161683,180624.052307,195158.159561,211730.427000,2.262089e+05,2.395141e+05,2.395141e+05,2.703884e+05,3.319213e+05,3.648160e+05
6,"Chicago, IL",225717.587667,246012.887969,255288.262533,250497.663970,223765.066690,201582.842850,186465.252079,170878.653276,167263.694859,...,196689.317741,206636.520325,216358.817461,228031.162400,2.352609e+05,2.401632e+05,2.401632e+05,2.564646e+05,2.813578e+05,2.951014e+05
7,"Columbus, OH",151478.699343,155521.101439,156101.632518,153107.763322,147981.738895,142587.867648,139393.754637,132484.884654,132411.504985,...,150138.939914,161965.997787,170510.582980,183409.689425,1.969140e+05,2.110286e+05,2.110286e+05,2.371701e+05,2.644115e+05,2.905528e+05
8,"Dallas, TX",147163.845871,149820.397881,154261.606352,155765.405497,147353.907583,148283.675091,143571.884621,139809.676373,147788.355192,...,173311.494415,193933.775162,215245.091553,235773.777741,2.488064e+05,2.564281e+05,2.564281e+05,2.782707e+05,3.362704e+05,3.734008e+05
9,"Denver, CO",245011.192907,249682.287547,248671.777039,242442.365210,231580.699226,230834.659840,225020.106175,219535.375036,238259.934286,...,289884.098646,331333.309212,363470.154715,390851.768914,4.129090e+05,4.300309e+05,4.300309e+05,4.701503e+05,5.523323e+05,5.749119e+05


In [366]:
uniquecities = housedata["City"].unique()
df_income_cr = df_income[df_income["City"].isin(uniquecities)]
df_income_cr.reset_index(inplace=True)
df_income_cr= df_income_cr.drop(columns=['index'], errors='ignore')
df_unem_cr = df_unem[df_unem["City"].isin(uniquecities)]
df_unem_cr.reset_index(inplace=True)
df_unem_cr= df_unem_cr.drop(columns=['index'], errors='ignore')
df_age_cr = df_age[df_age["City"].isin(uniquecities)]
df_age_cr.reset_index(inplace=True)
df_age_cr= df_age_cr.drop(columns=['index'], errors='ignore')
df_geomob_cr = df_geomob[df_geomob["City"].isin(uniquecities)]
df_geomob_cr.reset_index(inplace=True)
df_geomob_cr= df_geomob_cr.drop(columns=['index'], errors='ignore')

In [368]:
df_income_cr.to_csv('../data/income_31cities.csv')
df_unem_cr.to_csv('../data/unemployment_31cities.csv')
df_age_cr.to_csv('../data/medianage_31cities.csv')
df_geomob_cr.to_csv('../data/geomobility_31cities.csv')
housedata.to_csv('../data/houseprice_31cities.csv')

In [370]:
truckprices = pd.read_csv("../data/truckindex.csv")

In [371]:
truckprices

,2005,1.244
0,2006,1.2870
1,2007,1.2590
2,2008,1.1280
3,2009,0.9245
4,2010,1.0315
5,2011,1.1095
6,2012,1.1225
7,2013,1.1115
8,2014,1.1535
9,2015,1.1095
